In [1]:
import numpy as np
import os
import pandas as pd
import scipy.io as sio
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, auc
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.decomposition import SparsePCA

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp

from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, permutation_test_score
from sklearn import datasets

In [2]:
sns.set_style("whitegrid")
cwd = os.path.dirname(os.getcwd())
data_dir = os.path.join(os.path.dirname(os.getcwd()), 'data')
doc_dir = os.path.join(os.path.dirname(os.getcwd()), 'docs')

In [3]:
sfnc_pairs = sio.loadmat(os.path.join(data_dir,'sfnc_pairs.mat'))
sfnc_corr_pairs = sfnc_pairs['fnc_corrs']

In [4]:
demographics = pd.read_excel(os.path.join(data_dir, '20160420_vcalhoun_rest_demography_cag_info_new.xls'))

In [5]:
X = sfnc_corr_pairs
y = demographics.cap_d_group_id2.values

In [64]:
# HD diagnosed vs. Controls: Original group sizes (unbalanced: 23 diagnosed, 78 controls)
X = sfnc_corr_pairs
y_diagnosis = demographics["visit_diagnosis_ID"].values
X_diagnosed = X[y_diagnosis==1]
X_controls = X[y==0]

X_HD_diagnosed = pd.concat([pd.DataFrame(X_diagnosed), pd.DataFrame(X_controls)], axis=0)
y_HD_diagnosed = np.append(np.ones(np.shape(X_diagnosed)[0]), np.zeros(np.shape(X_controls)[0]))
log = LogisticRegression(penalty="l2", C=1, solver="liblinear")

cv = StratifiedKFold(5)

score, permutation_scores, pvalue = permutation_test_score(
                log, X_HD_diagnosed, y_HD_diagnosed, scoring="accuracy", cv=cv, n_permutations=1000, n_jobs=1)

print("Classification score %s (pvalue : %s)" % (score, pvalue))

Classification score 0.8025062656641604 (pvalue : 0.01098901098901099)


In [36]:
# HD diagnosed vs. Controls: Permutation tests for 100 random downsamplings of the control group
X = sfnc_corr_pairs
y_diagnosis = demographics["visit_diagnosis_ID"].values
X_diagnosed = X[y_diagnosis==1]
X_controls = X[y==0]
all_scores = np.zeros(100)
all_pvalue = np.zeros(100)
all_permutation_scores = np.zeros((100,1000))

for n in range(0,100,1):
    #random subset to establish group balance
    control_subset = np.random.choice(range(0,78,1),np.sum(y_diagnosis), replace=False)
    X_controls_subset = X_controls[control_subset].copy()
    # concatenate patients and controls
    X_HD_diagnosed = pd.concat([pd.DataFrame(X_diagnosed), pd.DataFrame(X_controls_subset)], axis=0)
    y_HD_diagnosed = np.append(np.ones(np.sum(y_diagnosis)), np.zeros(np.sum(y_diagnosis)))
    log = LogisticRegression(penalty="l2", C=1, solver="liblinear")

    cv = StratifiedKFold(5)

    score, permutation_scores, pvalue = permutation_test_score(
                log, X_HD_diagnosed, y_HD_diagnosed, scoring="accuracy", cv=cv, n_permutations=1000, n_jobs=1)
    all_scores[n] = score
    all_pvalue[n] = pvalue
    all_permutation_scores[n] = permutation_scores 
    print("Classification score %s (pvalue : %s)" % (score, pvalue))

Classification score 0.73 (pvalue : 0.011988011988011988)
Classification score 0.62 (pvalue : 0.1008991008991009)
Classification score 0.5599999999999999 (pvalue : 0.2777222777222777)
Classification score 0.72 (pvalue : 0.007992007992007992)
Classification score 0.675 (pvalue : 0.027972027972027972)
Classification score 0.6 (pvalue : 0.16583416583416583)
Classification score 0.63 (pvalue : 0.07292707292707293)
Classification score 0.74 (pvalue : 0.002997002997002997)
Classification score 0.655 (pvalue : 0.04995004995004995)
Classification score 0.705 (pvalue : 0.011988011988011988)
Classification score 0.8 (pvalue : 0.000999000999000999)
Classification score 0.735 (pvalue : 0.001998001998001998)
Classification score 0.615 (pvalue : 0.08791208791208792)
Classification score 0.5700000000000001 (pvalue : 0.21678321678321677)
Classification score 0.67 (pvalue : 0.030969030969030968)
Classification score 0.62 (pvalue : 0.08791208791208792)
Classification score 0.625 (pvalue : 0.074925074925

In [41]:
np.sum(all_pvalue<0.05)

61

In [66]:
# Late HD vs. Controls: Original group sizes (unbalanced: 57 near, 78 controls)
X = sfnc_corr_pairs
demographics["HD_near"] =  (demographics.cap_d_group=="high")*1

X_near = X[demographics["HD_near"]==1]
X_controls = X[y==0]

X_HD_near = pd.concat([pd.DataFrame(X_near), pd.DataFrame(X_controls)], axis=0)
y_HD_near = np.append(np.ones(np.shape(X_near)[0]), np.zeros(np.shape(X_controls)[0]))
log = LogisticRegression(penalty="l2", C=1, solver="liblinear")

cv = StratifiedKFold(5)

score, permutation_scores, pvalue = permutation_test_score(
                log, X_HD_near, y_HD_near, scoring="accuracy", cv=cv, n_permutations=1000, n_jobs=1)

print("Classification score %s (pvalue : %s)" % (score, pvalue))

Classification score 0.6746908602150536 (pvalue : 0.000999000999000999)


In [42]:
# Late HD vs. Controls: Permutation tests for 100 random downsamplings of the control group
X = sfnc_corr_pairs
demographics["HD_near"] =  (demographics.cap_d_group=="high")*1

X_near = X[demographics["HD_near"]==1]
# Only those Late HD pateints that have not been diagnosed yet
X_near = X_near[y_diagnosis[demographics["HD_near"]==1]==0]
print("There are %f Late HD patients" %np.shape(X_near)[0])
X_controls = X[y==0]
all_scores_late = np.zeros(100)
all_pvalue_late = np.zeros(100)
all_permutation_scores_late = np.zeros((100,1000))

for n in range(0,100,1):
    control_subset = np.random.choice(range(0,78,1), np.shape(X_near)[0], replace=False)
    X_controls_subset = X_controls[control_subset]
    y_near = np.ones(np.shape(X_near)[0])
    y_controls = np.zeros(np.shape(X_controls_subset)[0])
    # concatenate HD_near and controls
    X_HD_near = pd.concat([pd.DataFrame(X_near), pd.DataFrame(X_controls_subset)], axis=0)
    y_HD_near = np.append(y_near, y_controls)

    log = LogisticRegression(penalty="l2", C=1, solver="liblinear")
    cv = StratifiedKFold(5)

    score, permutation_scores, pvalue = permutation_test_score(
        log, X_HD_near, y_HD_near, scoring="accuracy", cv=cv, n_permutations=1000, n_jobs=1)
    all_scores_late[n] = score
    all_pvalue_late[n] = pvalue
    all_permutation_scores_late[n] = permutation_scores 
    print("Classification score %s (pvalue : %s)" % (score, pvalue))

There are 57.000000 Late HD patients
Classification score 0.7015151515151514 (pvalue : 0.000999000999000999)
Classification score 0.6227272727272727 (pvalue : 0.00999000999000999)
Classification score 0.6075757575757577 (pvalue : 0.03296703296703297)
Classification score 0.6477272727272727 (pvalue : 0.006993006993006993)
Classification score 0.5628787878787879 (pvalue : 0.13986013986013987)
Classification score 0.5272727272727273 (pvalue : 0.3106893106893107)
Classification score 0.6424242424242423 (pvalue : 0.01098901098901099)
Classification score 0.5984848484848484 (pvalue : 0.055944055944055944)
Classification score 0.6045454545454545 (pvalue : 0.04495504495504495)
Classification score 0.5992424242424242 (pvalue : 0.04495504495504495)
Classification score 0.5734848484848485 (pvalue : 0.1038961038961039)
Classification score 0.678030303030303 (pvalue : 0.001998001998001998)
Classification score 0.5719696969696969 (pvalue : 0.1108891108891109)
Classification score 0.5393939393939393 

In [43]:
np.sum(all_pvalue_late<0.05)

73

In [67]:
# Early HD vs. Controls: Original group sizes (unbalanced: 57 near, 78 controls)
X = sfnc_corr_pairs
demographics["HD_near"] =  (demographics.cap_d_group=="low")*1

X_near = X[demographics["HD_near"]==1]
X_controls = X[y==0]

X_HD_near = pd.concat([pd.DataFrame(X_near), pd.DataFrame(X_controls)], axis=0)
y_HD_near = np.append(np.ones(np.shape(X_near)[0]), np.zeros(np.shape(X_controls)[0]))
log = LogisticRegression(penalty="l2", C=1, solver="liblinear")

cv = StratifiedKFold(5)

score, permutation_scores, pvalue = permutation_test_score(
                log, X_HD_near, y_HD_near, scoring="accuracy", cv=cv, n_permutations=1000, n_jobs=1)

print("Classification score %s (pvalue : %s)" % (score, pvalue))

Classification score 0.5823703703703704 (pvalue : 0.14085914085914086)


In [53]:
# Early HD vs. Controls: Permutation tests for 100 random downsamplings of the control group
X = sfnc_corr_pairs
demographics["HD_far"] =  (demographics.cap_d_group=="low")*1

X_far = X[demographics["HD_far"]==1]
# Only those early HD pateints that have not been diagnosed yet
X_far = X_far[y_diagnosis[demographics["HD_far"]==1]==0]
print("There are %f Early HD patients" %np.shape(X_near)[0])
X_controls = X[y==0]
all_scores_early = np.zeros(100)
all_pvalue_early = np.zeros(100)
all_permutation_scores_early = np.zeros((100,1000))

for n in range(0,100,1):
    controls_subset = np.random.choice(range(0,78,1), np.shape(X_far)[0], replace=False)
    X_controls_subset = X_controls[controls_subset]
    y_far = np.ones(np.shape(X_far)[0])
    y_controls = np.zeros(np.shape(X_controls_subset)[0])
    # concatenate HD_near and controls
    X_HD_far = pd.concat([pd.DataFrame(X_far), pd.DataFrame(X_controls_subset)], axis=0)
    y_HD_far = np.append(y_far, y_controls)

    log = LogisticRegression(penalty="l2", C=1, solver="liblinear")
    cv = StratifiedKFold(5)

    score, permutation_scores, pvalue = permutation_test_score(
        log, X_HD_far, y_HD_far, scoring="accuracy", cv=cv, n_permutations=1000, n_jobs=1)
    all_scores_early[n] = score
    all_pvalue_early[n] = pvalue
    all_permutation_scores_early[n] = permutation_scores 
    print("Classification score %s (pvalue : %s)" % (score, pvalue))

There are 57.000000 Early HD patients
Classification score 0.6018181818181818 (pvalue : 0.04295704295704296)
Classification score 0.5636363636363636 (pvalue : 0.17282717282717283)
Classification score 0.5272727272727272 (pvalue : 0.3166833166833167)
Classification score 0.6672727272727274 (pvalue : 0.003996003996003996)
Classification score 0.649090909090909 (pvalue : 0.004995004995004995)
Classification score 0.6209090909090909 (pvalue : 0.029970029970029972)
Classification score 0.5745454545454545 (pvalue : 0.13386613386613386)
Classification score 0.6190909090909091 (pvalue : 0.03196803196803197)
Classification score 0.5845454545454545 (pvalue : 0.0919080919080919)
Classification score 0.61 (pvalue : 0.03996003996003996)
Classification score 0.5563636363636364 (pvalue : 0.22877122877122877)
Classification score 0.6190909090909091 (pvalue : 0.023976023976023976)
Classification score 0.659090909090909 (pvalue : 0.004995004995004995)
Classification score 0.5827272727272728 (pvalue : 0.

In [54]:
np.sum(all_pvalue_early<0.05)

36

In [68]:
# Medium HD vs. Controls: Original group sizes (unbalanced: 57 near, 78 controls)
X = sfnc_corr_pairs
demographics["HD_near"] =  (demographics.cap_d_group=="med")*1

X_near = X[demographics["HD_near"]==1]
X_controls = X[y==0]

X_HD_near = pd.concat([pd.DataFrame(X_near), pd.DataFrame(X_controls)], axis=0)
y_HD_near = np.append(np.ones(np.shape(X_near)[0]), np.zeros(np.shape(X_controls)[0]))
log = LogisticRegression(penalty="l2", C=1, solver="liblinear")

cv = StratifiedKFold(5)

score, permutation_scores, pvalue = permutation_test_score(
                log, X_HD_near, y_HD_near, scoring="accuracy", cv=cv, n_permutations=1000, n_jobs=1)

print("Classification score %s (pvalue : %s)" % (score, pvalue))

Classification score 0.5351111111111111 (pvalue : 0.4275724275724276)


In [57]:
# Medium HD vs. Controls: Permutation tests for 100 random downsamplings of the control group
X = sfnc_corr_pairs
demographics["HD_far"] =  (demographics.cap_d_group=="med")*1

X_far = X[demographics["HD_far"]==1]
# Only those early HD pateints that have not been diagnosed yet
X_far = X_far[y_diagnosis[demographics["HD_far"]==1]==0]
print("There are %f Early HD patients" %np.shape(X_far)[0])
X_controls = X[y==0]
all_scores_med = np.zeros(100)
all_pvalue_med = np.zeros(100)
all_permutation_scores_med = np.zeros((100,1000))

for n in range(0,100,1):
    controls_subset = np.random.choice(range(0,78,1), np.shape(X_far)[0], replace=False)
    X_controls_subset = X_controls[controls_subset]
    y_far = np.ones(np.shape(X_far)[0])
    y_controls = np.zeros(np.shape(X_controls_subset)[0])
    # concatenate HD_near and controls
    X_HD_far = pd.concat([pd.DataFrame(X_far), pd.DataFrame(X_controls_subset)], axis=0)
    y_HD_far = np.append(y_far, y_controls)

    log = LogisticRegression(penalty="l2", C=1, solver="liblinear")
    cv = StratifiedKFold(5)

    score, permutation_scores, pvalue = permutation_test_score(
        log, X_HD_far, y_HD_far, scoring="accuracy", cv=cv, n_permutations=1000, n_jobs=1)
    all_scores_med[n] = score
    all_pvalue_med[n] = pvalue
    all_permutation_scores_med[n] = permutation_scores 
    print("Classification score %s (pvalue : %s)" % (score, pvalue))

There are 49.000000 Early HD patients
Classification score 0.5922222222222222 (pvalue : 0.08491508491508491)
Classification score 0.44222222222222224 (pvalue : 0.8291708291708292)
Classification score 0.5833333333333333 (pvalue : 0.0979020979020979)
Classification score 0.5622222222222223 (pvalue : 0.17182817182817184)
Classification score 0.5144444444444445 (pvalue : 0.3946053946053946)
Classification score 0.5122222222222222 (pvalue : 0.3956043956043956)
Classification score 0.5533333333333333 (pvalue : 0.22377622377622378)
Classification score 0.4355555555555556 (pvalue : 0.8431568431568431)
Classification score 0.5077777777777778 (pvalue : 0.46253746253746253)
Classification score 0.5644444444444445 (pvalue : 0.1258741258741259)
Classification score 0.5433333333333332 (pvalue : 0.21578421578421578)
Classification score 0.5333333333333333 (pvalue : 0.3016983016983017)
Classification score 0.5955555555555556 (pvalue : 0.07092907092907093)
Classification score 0.5222222222222223 (pval

In [59]:
np.sum(all_pvalue_med<0.05)

7